In [200]:
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px 
import numpy as np 
%matplotlib inline
pd.set_option('display.float_format', lambda x:'%.3f' % x)
np.printoptions(suppress=True, precision=3)

from catboost import CatBoostRegressor 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import root_mean_squared_error, r2_score
from scipy.stats import chi2_contingency

from colorama import Fore, Style, Back

import optuna

### Carregando dados

In [275]:
df_clientes = pd.read_csv(r'datasets\dataset_clientes.csv')


In [276]:
# Estrutura do Data set de clientes atuais
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID_Cliente                   1000 non-null   object 
 1   Nome                         1000 non-null   object 
 2   Idade                        1000 non-null   int64  
 3   Viagens                      1000 non-null   int64  
 4   Restaurantes                 1000 non-null   int64  
 5   Entretenimento               1000 non-null   int64  
 6   Cashback                     1000 non-null   int64  
 7   Compras online               1000 non-null   int64  
 8   Farmácias                    1000 non-null   int64  
 9   Programas de Milhagem        1000 non-null   int64  
 10  Postos de Combustível        1000 non-null   int64  
 11  Mercados                     1000 non-null   int64  
 12  Cidade                       1000 non-null   object 
 13  Cargo              

In [277]:
# 10 primeiros registos do data frame de clientes
df_clientes.head(10)

,ID_Cliente,Nome,Idade,Viagens,Restaurantes,Entretenimento,Cashback,Compras online,Farmácias,Programas de Milhagem,...,Pessoas em Casa,Moradia,Possui Carro,Renda,Investimentos,Ativos,Dívidas,Probabilidade Inadimplencia,Score,Principal Cartão
0,bc3749c2-932f-462b-a71e-d1a352510f67,Nicole Lopes,32,5,4,2,4,5,3,1,...,2,Próprio,True,34118,631262,55649,294254,0.000,232,Rocketseat Mastercard Travel Elite
1,981dffc4-087e-4a99-bbac-062944f9b5a7,Kevin Teixeira,42,1,4,5,1,4,1,5,...,2,Alugado,True,7338,376417,1777324,180331,87.000,27,Rocketseat Mastercard Life Basic
2,242f4e94-7f2a-4557-a27a-9227cca55e83,Augusto Sá,67,4,3,4,3,2,3,3,...,6,Alugado,True,19965,83667,1793731,122049,78.450,57,Rocketseat Mastercard Travel Basic
3,e6f16b89-d9bf-4448-9d08-65957cdc1fb3,Ágatha Moura,47,2,3,1,2,1,3,4,...,6,Próprio,False,25867,283060,1940685,335544,85.210,32,Rocketseat Visa Travel Basic
4,79b7f5b0-e754-4cd9-9150-ed5913fef23a,João Rios,27,3,5,4,5,4,3,2,...,1,Alugado,True,18359,146413,517215,390588,89.120,0,Rocketseat Visa Life Basic
5,c403b04d-6f51-4374-b367-1adce70cdafb,Juliana Sales,56,5,3,5,1,1,5,3,...,4,Alugado,False,5163,403457,800313,312417,82.440,0,Rocketseat Mastercard Travel Basic
6,fd42f59f-18f3-428a-8e04-83263518946a,Sra. Brenda Almeida,50,5,5,2,2,3,2,5,...,1,Alugado,False,7973,912804,1311349,156471,78.360,74,Rocketseat Visa Travel Basic
7,e1a94ba9-8c52-4bc0-a7c5-6ea6ab430f6e,Lavínia Rios,33,1,4,1,5,2,2,4,...,5,Alugado,False,4796,252572,1841319,297458,93.470,4,Rocketseat Visa Life Basic
8,bb3e64ba-c46b-4ce7-a915-a5ac5d25fb3e,Rhavi Rezende,60,1,2,2,1,3,1,5,...,1,Alugado,True,43590,391559,918762,471656,76.120,43,Rocketseat Mastercard Travel Basic
9,11278877-2221-4c1c-842a-d8fe968f295e,Otto Correia,75,3,4,2,5,2,5,5,...,2,Alugado,True,3058,901393,692958,37149,59.740,135,Rocketseat Visa Life Select


In [278]:
df_clientes.isna().sum()

ID_Cliente                     0
Nome                           0
Idade                          0
Viagens                        0
Restaurantes                   0
Entretenimento                 0
Cashback                       0
Compras online                 0
Farmácias                      0
Programas de Milhagem          0
Postos de Combustível          0
Mercados                       0
Cidade                         0
Cargo                          0
Estado Civil                   0
Tempo na Empresa               0
Pessoas em Casa                0
Moradia                        0
Possui Carro                   0
Renda                          0
Investimentos                  0
Ativos                         0
Dívidas                        0
Probabilidade Inadimplencia    0
Score                          0
Principal Cartão               0
dtype: int64

Dataset de clientes não possui entradas nulas

### Engenharia de Features

In [279]:
# Removendo colunas de linhas unicas para cada entrada, e a variavel target Score
df_clientes.drop(columns=['ID_Cliente', 'Nome', 'Score', 'Principal Cartão'], axis=1, inplace=True)

In [280]:
df_clientes['Possui Carro'] = df_clientes['Possui Carro'].astype(int)

In [281]:
# Verificar se há features categoricas de valores unicos

categorical_features = df_clientes.select_dtypes(include=['object']).columns

for col in categorical_features:
    print(f"Coluna: {col} possui os possiveis valores:\n{df_clientes[col].unique()}\n")

Coluna: Cidade possui os possiveis valores:
['Brasília' 'Campo Grande' 'Goiânia' 'Porto Alegre' 'Fortaleza' 'Curitiba'
 'Manaus' 'Florianópolis' 'Rio de Janeiro' 'São Paulo' 'Belém'
 'Belo Horizonte' 'Recife' 'Vitória' 'Salvador']

Coluna: Cargo possui os possiveis valores:
['Engenheiro agrônomo' 'Psicólogo' 'Intérprete de Bíblias'
 'Engenheiro de projetos' 'Cantor' 'Tecelão' 'Babysitter' 'Astronauta'
 'Ferramenteiro' 'Garimpeiro' 'Contábil' 'Assessor de imprensa'
 'Lanterneiro' 'Intérprete e tradutor de língua de sinais' 'Meeiro'
 'Lavador' 'Cancerologista ou Oncologista' 'Ambientalista' 'Almoxarife'
 'Jogador de basquete' 'Barman' 'Escoteiro' 'Gerente de riscos em seguros'
 'Parteira tradicional' 'Produtor de multimídia' 'Restaurador'
 'Office-boy' 'Estampador de tecidos' 'Biomédico' 'Geógrafo' 'Coronel'
 'Químico' 'Servente-de-obras' 'Gestor ambiental'
 'Tecnólogo em geoprocessamento' 'Instalador de linha telefônica'
 'Viveirista' 'Toxicologista' 'Pescador' 'Coloproctologista'
 'Méd

In [282]:
# Verificar se há features numericas de valores unicos

numeric_features = df_clientes.select_dtypes(include=['number']).columns

for col in numeric_features:
    print(f"Coluna: {col} possui os possiveis valores:\n{df_clientes[col].unique()}\n")

Coluna: Idade possui os possiveis valores:
[32 42 67 47 27 56 50 33 60 75 40 73 21 74 53 68 43 46 57 44 35 29 78 54
 70 69 23 64 18 76 48 80 71 59 66 24 72 22 41 79 19 31 38 58 39 20 77 25
 65 63 62 37 28 36 61 30 49 45 52 34 51 55 26]

Coluna: Viagens possui os possiveis valores:
[5 1 4 2 3]

Coluna: Restaurantes possui os possiveis valores:
[4 3 5 2 1]

Coluna: Entretenimento possui os possiveis valores:
[2 5 4 1 3]

Coluna: Cashback possui os possiveis valores:
[4 1 3 2 5]

Coluna: Compras online possui os possiveis valores:
[5 4 2 1 3]

Coluna: Farmácias possui os possiveis valores:
[3 1 5 2 4]

Coluna: Programas de Milhagem possui os possiveis valores:
[1 5 3 4 2]

Coluna: Postos de Combustível possui os possiveis valores:
[2 3 5 4 1]

Coluna: Mercados possui os possiveis valores:
[4 5 3 1 2]

Coluna: Tempo na Empresa possui os possiveis valores:
[ 4 18 19  6  1  5 15  0 13 12 10 16  3 11  9  2  8 14  7 20 17]

Coluna: Pessoas em Casa possui os possiveis valores:
[2 6 1 4 5 3]

Co

In [283]:
# Estatisiticas das variaveis numericas

df_clientes.describe()

,Idade,Viagens,Restaurantes,Entretenimento,Cashback,Compras online,Farmácias,Programas de Milhagem,Postos de Combustível,Mercados,Tempo na Empresa,Pessoas em Casa,Possui Carro,Renda,Investimentos,Ativos,Dívidas,Probabilidade Inadimplencia
count,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000,1000.000
mean,49.432,2.986,3.023,3.073,2.985,2.999,2.979,3.003,3.091,3.001,10.144,3.568,0.505,25556.416,485209.921,986604.889,248727.312,66.727
std,18.320,1.403,1.418,1.392,1.389,1.423,1.384,1.460,1.432,1.439,6.074,1.747,0.500,14468.363,289308.842,574234.666,143546.398,22.381
min,18.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,1.000,0.000,1000.000,89.000,2538.000,31.000,0.000
25%,34.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,5.000,2.000,0.000,12695.250,225495.500,511514.250,122040.000,60.748
50%,49.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,10.000,4.000,1.000,25494.500,478102.500,957918.500,251886.500,74.290
75%,66.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,15.000,5.000,1.000,38384.750,733778.750,1492880.750,376115.250,81.005
max,80.000,5.000,5.000,5.000,5.000,5.000,5.000,5.000,5.000,5.000,20.000,6.000,1.000,49996.000,997209.000,1998481.000,499732.000,98.210


### EDA

In [243]:
# Data frame para EDA
df_eda = df_clientes.copy()

#### Bucket

In [284]:
# Mapeamento de cidades por regiões, afim de melhorar a visualização dos gráficos
map_regioes = {
    "Brasília": "Centro-Oeste",
    "Campo Grande": "Centro-Oeste",
    "Goiânia": "Centro-Oeste",

    "São Paulo": "Sudeste",
    "Rio de Janeiro": "Sudeste",
    "Belo Horizonte": "Sudeste",
    "Vitória": "Sudeste",

    "Curitiba": "Sul",
    "Florianópolis": "Sul",
    "Porto Alegre": "Sul",

    "Salvador": "Nordeste",
    "Recife": "Nordeste",
    "Fortaleza": "Nordeste",

    "Belém": "Norte",
    "Manaus": "Norte"
}

df_eda["Região"] = df_clientes["Cidade"].map(map_regioes)

In [285]:
# Bucketing Idade para melhorar a visualização dos gráficos
bins_idade = [10,20,30,40,50,60,70,80]
labels_idade = ['10-19','20-29','30-39','40-49','50-59','60-69','70+']
df_eda['Idade_range'] =pd.cut(x=df_clientes['Idade'], bins=bins_idade, labels=labels_idade, include_lowest=True)

In [286]:
# Bucketing Renda para melhorar a visualização dos gráficos
bins_renda = [0, 12000, 20000, 30000, 40000, 50000]
labels_renda = ['Muito Baixa', 'Baixa', 'Média', 'Alta', 'Muito Alta']

df_eda['Renda_range'] = pd.cut(df_clientes['Renda'], bins=bins_renda, labels=labels_renda, include_lowest=True)


In [287]:
# Bucketing Investimentos para melhorar a visualização dos gráficos
bins_invest = [0, 200000, 400000, 600000, 800000, 1000000]
labels_invest = ['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']

df_eda['Investimentos_range'] = pd.cut(df_clientes['Investimentos'], bins=bins_invest, labels=labels_invest, include_lowest=True)


In [288]:
# Bucketing Ativos para melhorar a visualização dos gráficos
bins_ativos = [0, 500000, 900000, 1300000, 1700000, 2000000]
labels_ativos = ['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']

df_eda['Ativos_range'] = pd.cut(df_clientes['Ativos'], bins=bins_ativos, labels=labels_ativos, include_lowest=True)


In [289]:
# Bucketing Dividas para melhorar a visualização dos gráficos
bins_dividas = [0, 120000, 200000, 300000, 400000, 500000]
labels_dividas = ['Muito Baixa', 'Baixa', 'Média', 'Alta', 'Muito Alta']

df_eda['Dividas_range'] = pd.cut(df_clientes['Dívidas'], bins=bins_dividas, labels=labels_dividas, include_lowest=True)

In [290]:
# Visualizar primeiros registros com os ranges
df_eda.head(10)

,Idade,Viagens,Restaurantes,Entretenimento,Cashback,Compras online,Farmácias,Programas de Milhagem,Postos de Combustível,Mercados,...,Investimentos,Ativos,Dívidas,Probabilidade Inadimplencia,Região,Idade_range,Renda_range,Investimentos_range,Ativos_range,Dividas_range
0,32,5,4,2,4,5,3,1,2,4,...,631262,55649,294254,0.000,Centro-Oeste,30-39,Alta,Alto,Muito Baixo,Média
1,42,1,4,5,1,4,1,5,3,5,...,376417,1777324,180331,87.000,Centro-Oeste,40-49,Muito Baixa,Baixo,Muito Alto,Baixa
2,67,4,3,4,3,2,3,3,2,3,...,83667,1793731,122049,78.450,Centro-Oeste,60-69,Baixa,Muito Baixo,Muito Alto,Baixa
3,47,2,3,1,2,1,3,4,3,1,...,283060,1940685,335544,85.210,Sul,40-49,Média,Baixo,Muito Alto,Alta
4,27,3,5,4,5,4,3,2,2,5,...,146413,517215,390588,89.120,Nordeste,20-29,Baixa,Muito Baixo,Baixo,Alta
5,56,5,3,5,1,1,5,3,3,1,...,403457,800313,312417,82.440,Sul,50-59,Muito Baixa,Médio,Baixo,Alta
6,50,5,5,2,2,3,2,5,5,1,...,912804,1311349,156471,78.360,Norte,40-49,Muito Baixa,Muito Alto,Alto,Baixa
7,33,1,4,1,5,2,2,4,5,2,...,252572,1841319,297458,93.470,Sul,30-39,Muito Baixa,Baixo,Muito Alto,Média
8,60,1,2,2,1,3,1,5,5,2,...,391559,918762,471656,76.120,Norte,50-59,Muito Alta,Baixo,Médio,Muito Alta
9,75,3,4,2,5,2,5,5,4,2,...,901393,692958,37149,59.740,Sul,70+,Muito Baixa,Muito Alto,Baixo,Muito Baixa


#### Boxplots

In [291]:
# Boxplot de Probabilidade Inadimplencia por Região

fig = px.box(
    df_eda,
    x='Região',
    y='Probabilidade Inadimplencia',
    title='Probabilidade de Inadimplência por Região do Brasil',
    category_orders={'Região': ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul']}
)

fig.show()

Por esse boxplot temos que a probabilidade de inadimplência por região possui medianas bem próximas, implicando que não há uma forte correlação entre essas variáveis

In [292]:
# Boxplot de Probabilidade Inadimplencia por Idade
fig = px.box(df_eda, 
             x='Idade_range', 
             y='Probabilidade Inadimplencia', 
             title=f'Box plot de Idade com Probabilidade Inadimplencia',
             category_orders={'Idade_range': labels_idade})
fig.show()

Por esse boxplot temos que a mediana de probabilidade de inadimplência por ranges de idade vai diminuindo ao aumentar a idade, evento previsto no mundo real, e mostra que essas duas variaveis possui alguma correlação.

In [293]:
# Boxplot de Probabilidade Inadimplencia por Renda
fig = px.box(df_eda, 
             x='Renda_range',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Renda com Probabilidade Inadimplencia',
             category_orders={'Renda_range': labels_renda})
fig.show()

Por esse boxplot temos que a mediana de probabilidade de inadimplência por ranges de renda vai diminuindo ao aumentar a renda,mostrando que essas duas variaveis possui alguma correlação.

In [294]:
# Boxplot de Probabilidade Inadimplencia por Investimentos
fig = px.box(df_eda,
             x='Investimentos_range',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Investimentos com Probabilidade Inadimplencia',
             category_orders={'Investimentos_range': labels_invest})
fig.show()

Por esse boxplot temos que a mediana de probabilidade de inadimplência por ranges de investimentos vai diminuindo ao aumentar a investimentos,mostrando que essas duas variaveis possui alguma correlação.

In [295]:
# Boxplot de Probabilidade Inadimplencia por Ativos
fig = px.box(df_eda, 
             x='Ativos_range',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Ativos com Probabilidade Inadimplencia',
             category_orders={'Ativos_range': labels_ativos})
fig.show()

Por esse boxplot temos que quanto mais ativos  uma pessoa possui, maior sera sua probabilidade de inadimplencia, mostrando que essas variáveis possuem alguma correlação.


In [296]:
# Boxplot de Probabilidade Inadimplencia por Dividas
fig = px.box(df_eda,
             x='Dividas_range',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Dividas com Probabilidade Inadimplencia',
             category_orders={'Dividas_range': labels_dividas})
fig.show()

Por esse boxplot temos que quanto mais dividas  uma pessoa possui, maior sera sua probabilidade de inadimplencia, mostrando que essas variáveis possuem alguma correlação.

In [297]:
labels_order_1_5 = ['1','2','3','4','5']

In [298]:
# Boxplot de Probabilidade Inadimplencia por Viagens
fig = px.box(df_eda,
             x='Viagens',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Viagens com Probabilidade Inadimplencia',
             category_orders={'Viagens': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de viagens de uma pessoa não influencia na sua probabilidade de inadimplencia

In [299]:
# Boxplot de Probabilidade Inadimplencia por Restaurantes
fig = px.box(df_eda,
             x='Restaurantes',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Restaurantes com Probabilidade Inadimplencia',
             category_orders={'Restaurantes': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de idas ao restaurante de uma pessoa não influencia na sua probabilidade de inadimplencia

In [300]:
# Boxplot de Probabilidade Inadimplencia por Mercados
fig = px.box(df_eda,
             x='Mercados',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Mercados com Probabilidade Inadimplencia',
             category_orders={'Mercados': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de idas ao mercado de uma pessoa não influencia na sua probabilidade de inadimplencia

In [301]:
# Boxplot de Probabilidade Inadimplencia por Entretenimento
fig = px.box(df_eda,
             x='Entretenimento',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Entretenimento com Probabilidade Inadimplencia',
             category_orders={'Entretenimento': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de idas a um evento de entretenimento de uma pessoa não influencia na sua probabilidade de inadimplencia

In [302]:
# Boxplot de Probabilidade Inadimplencia por Cashback
fig = px.box(df_eda,
             x='Cashback',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Cashback com Probabilidade Inadimplencia',
             category_orders={'Cashback': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de cashbacks de uma pessoa não influencia na sua probabilidade de inadimplencia

In [303]:
# Boxplot de Probabilidade Inadimplencia por Cashback
fig = px.box(df_eda,
             x='Compras online',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Compras online com Probabilidade Inadimplencia',
             category_orders={'Compras online': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de compras online de uma pessoa não influencia na sua probabilidade de inadimplencia

In [304]:
# Boxplot de Probabilidade Inadimplencia por Farmácias
fig = px.box(df_eda,
             x='Farmácias',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Farmácias com Probabilidade Inadimplencia',
             category_orders={'Farmácias': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de idas a farmacia de uma pessoa não influencia na sua probabilidade de inadimplencia

In [305]:
# Boxplot de Probabilidade Inadimplencia por Programas de Milhagem
fig = px.box(df_eda,
             x='Programas de Milhagem',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Programas de Milhagem com Probabilidade Inadimplencia',
             category_orders={'Programas de Milhagem': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de programas de milhagem de uma pessoa não influencia na sua probabilidade de inadimplencia

In [306]:
# Boxplot de Probabilidade Inadimplencia por Programas de Milhagem
fig = px.box(df_eda,
             x='Postos de Combustível',
             y='Probabilidade Inadimplencia',
             title=f'Box plot de Postos de Combustível com Probabilidade Inadimplencia',
             category_orders={'Postos de Combustível': labels_order_1_5})
fig.show()

Por essse boxplot temos que a quantidade de vezzes que uma pessoa abastece seu veículo não influencia na sua probabilidade de inadimplencia

#### Distribuições

In [307]:
for col in numeric_features:
    fig = px.histogram(df_clientes, x=col, title=f"Distribuição da Feature Númerica: {col}")
    fig.show()

Apartir desses gráficos visualizamos que as variáveis estao bem distribuidas com excessão da variavel target

#### Contagem de Variáveis Categoricas

In [308]:
categorical_features = df_clientes.select_dtypes(include=['object']).columns

for col in categorical_features:
    df_count = df_clientes[col].value_counts().reset_index()
    df_count.columns = ['Categoria', 'Contagem']
    df_count = df_count.sort_values(by='Contagem', ascending=True)
    fig = px.bar(df_count, x='Contagem', y='Categoria', orientation='h', title=f'Contagem por Categoria da {col}')
    fig.update_layout(
        height=800,
        width=600
    )
    fig.show()

#### Matriz de correlação

In [309]:
correlation_matrix = df_clientes.select_dtypes(include=['number']).corr()

In [310]:
fig = px.imshow(correlation_matrix, 
                color_continuous_scale='Viridis',
                title='Matriz de Correlação',
                zmin=-1,
                zmax=1)

fig.update_traces(text=correlation_matrix,
                  texttemplate='%{text:.1%}',
                  textfont=dict(size=9))

fig.update_layout(width=1200,
                  height= 800,
                  title_font=dict(size=14),
                  font=dict(size=10)
                  )

fig.show()

In [311]:
correlation_matrix['Probabilidade Inadimplencia']

Idade                         -0.220
Viagens                       -0.008
Restaurantes                   0.044
Entretenimento                -0.006
Cashback                       0.025
Compras online                -0.006
Farmácias                      0.011
Programas de Milhagem          0.011
Postos de Combustível          0.031
Mercados                       0.019
Tempo na Empresa               0.011
Pessoas em Casa               -0.019
Possui Carro                  -0.016
Renda                         -0.196
Investimentos                 -0.311
Ativos                         0.552
Dívidas                        0.270
Probabilidade Inadimplencia    1.000
Name: Probabilidade Inadimplencia, dtype: float64

Como as variaveis numericas Viagens, Restaurantes, Entretenimento, CashBack, Compras Online, Farmácias, Programs de Milhagem, Posto de Combustivel, Mercados, Tempo na empresa, Pessoas em casa e Possui carro suas correlações com a variavel target se aproximam de zero irei descarta-las da contrução do modelo

#### Teste de Hipótese

In [312]:
# Teste de Qui-Quadrado para variáveis categoricas

for cat in categorical_features:
    contingecy_table = pd.crosstab(df_clientes[cat], df_clientes['Probabilidade Inadimplencia'])
    chi2, p, dof, _ = chi2_contingency(contingecy_table)
    print(f"{Fora.RED if p < 0.05 else Fore.WHITE}"
          f"Teste de QUI-QUADRADO entre {cat} e Probabilidade Inadimplencia: p-valor = {p}")

Teste de QUI-QUADRADO entre Cidade e Probabilidade Inadimplencia: p-valor = 0.3840071939383141
Teste de QUI-QUADRADO entre Cargo e Probabilidade Inadimplencia: p-valor = 0.12537953599869553
Teste de QUI-QUADRADO entre Estado Civil e Probabilidade Inadimplencia: p-valor = 0.48352948196643847
Teste de QUI-QUADRADO entre Moradia e Probabilidade Inadimplencia: p-valor = 0.37283145381687693


Como p-value para as variaveis categoricas é maior que 0.05 entao elas são independentes, logo irei descarta-las no treinamento do modelo

In [314]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Idade                        1000 non-null   int64  
 1   Viagens                      1000 non-null   int64  
 2   Restaurantes                 1000 non-null   int64  
 3   Entretenimento               1000 non-null   int64  
 4   Cashback                     1000 non-null   int64  
 5   Compras online               1000 non-null   int64  
 6   Farmácias                    1000 non-null   int64  
 7   Programas de Milhagem        1000 non-null   int64  
 8   Postos de Combustível        1000 non-null   int64  
 9   Mercados                     1000 non-null   int64  
 10  Cidade                       1000 non-null   object 
 11  Cargo                        1000 non-null   object 
 12  Estado Civil                 1000 non-null   object 
 13  Tempo na Empresa   

In [315]:
df_clientes.drop(columns=['Cidade', 
                          'Cargo', 
                          'Estado Civil',
                          'Moradia',
                          'Viagens',
                          'Restaurantes',
                          'Entretenimento',
                          'Cashback',
                          'Compras online',
                          'Farmácias',
                          'Programas de Milhagem',
                          'Postos de Combustível',
                          'Mercados',
                          'Tempo na Empresa',
                          'Pessoas em Casa', 
                          'Possui Carro'], axis=1, inplace=True)

### Preparação dos Dados

In [ ]:
X = df_clientes.drop(columns=['Probabilidade Inadimplencia'], axis=1)
y = df_clientes['Probabilidade Inadimplencia']



In [317]:
categorical_features = X.select_dtypes(include=['object']).columns

### Treinamento do Modelo CatBoost

In [ ]:
# Funcao de otimizacao dos hiperparametros 
def objective(trial):

    params = {
        "iterations": trial.suggest_int("iterations", 300, 1500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 3, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "random_strength": trial.suggest_float("random_strength", 1e-5, 10, log=True),
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "cat_features": categorical_features.tolist(),
        "verbose": False
    }

    # CatBoost lida bem com splits aleatórios mesmo com poucos dados
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

    preds = model.predict(X_test)
    rmse = root_mean_squared_error(y_test, preds)

    return rmse


In [319]:
# Estudo do optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=60)

[I 2025-11-15 11:30:57,717] A new study created in memory with name: no-name-6dfeb946-cf8d-4c43-bcd4-d599f3c72908
[I 2025-11-15 11:31:00,650] Trial 0 finished with value: 3.7603774919634056 and parameters: {'iterations': 1369, 'learning_rate': 0.1416481802141313, 'depth': 3, 'l2_leaf_reg': 7.0659006697995554, 'random_strength': 0.00024162631613352842}. Best is trial 0 with value: 3.7603774919634056.
[I 2025-11-15 11:31:14,648] Trial 1 finished with value: 5.770821755721799 and parameters: {'iterations': 1376, 'learning_rate': 0.012603891449335809, 'depth': 9, 'l2_leaf_reg': 3.101886416640215, 'random_strength': 0.0010529168113554005}. Best is trial 0 with value: 3.7603774919634056.
[I 2025-11-15 11:31:16,509] Trial 2 finished with value: 4.171200965227395 and parameters: {'iterations': 612, 'learning_rate': 0.03644783766506412, 'depth': 6, 'l2_leaf_reg': 3.503086489607824, 'random_strength': 0.0007998848989898879}. Best is trial 0 with value: 3.7603774919634056.
[I 2025-11-15 11:31:18,

In [320]:
# Melhores parametros
print("\nBEST PARAMETERS:")
print(study.best_params)


BEST PARAMETERS:
{'iterations': 1169, 'learning_rate': 0.029394760408969343, 'depth': 4, 'l2_leaf_reg': 1.5444640682824762, 'random_strength': 5.507653385672015}


In [321]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [322]:
# Treinando melhor modelo
best_params = study.best_params
best_params["loss_function"] = "RMSE"
best_params["cat_features"] = categorical_features.tolist()

final_model = CatBoostRegressor(**best_params)
final_model.fit(X_train, y_train, verbose=False)

In [323]:
y_pred = final_model.predict(X_test)
y_pred = np.clip(y_pred, 0, 100)

In [324]:
y_pred

array([75.70025613, 77.47837783, 88.32671969, 65.25554026, 85.11601261,
       59.29924605, 70.79189574, 87.92068297, 71.75981599, 79.66105957,
       85.91081978, 67.33207655, 82.64780567, 77.12767186, 57.88943284,
       83.25663939, 78.74969943, 77.66421633, 58.2904571 , 86.14033844,
        3.754769  , 64.57148399, 87.77015863,  0.        , 68.85900265,
       68.9822351 , 25.40682199,  0.        , 77.75688577, 65.16000441,
       39.73214979, 72.77487935, 67.67702863, 77.46413396, 69.23004985,
       66.59257331, 85.95210976, 78.79385402, 59.58301375, 79.16367557,
       72.59913787, 76.27095982, 77.50938133, 53.66552009, 88.6177356 ,
       35.05634276, 72.63680337, 40.03150285, 60.51191604, 54.54753532,
       87.24525188, 69.55554182,  8.3886247 , 88.41920452, 78.05984047,
       47.45098717, 57.5353058 , 83.29202594, 64.36146166, 76.01685371,
       73.36884179, 76.12498881, 80.23010342, 84.49870306, 72.17384939,
       81.01161175, 73.67032537, 78.21996801, 65.93859834, 40.10

In [325]:
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [326]:
# Metricas
print(f'RMSE: {rmse}\nR2SCORE: {r2}')

RMSE: 2.6780616519683975
R2SCORE: 0.984299704961742


Com o modelo CatBoost, que é especialmente eficaz para datasets pequenos, realizei uma otimização de hiperparâmetros para maximizar sua performance. O modelo obteve um RMSE de aproximadamente 2.68, o que representa um erro médio muito baixo considerando que a probabilidade varia de 0 a 100. Além disso, o modelo alcançou um R² de cerca de 0,98, indicando que ele consegue explicar quase 98% da variabilidade dos dados. Esses resultados demonstram que o modelo não apenas prediz com alta precisão, mas também captura muito bem os padrões presentes no conjunto de dados.

### Predição em Prospect

In [327]:
df_prospect = pd.read_csv(r'datasets\dataset_prospect.csv')

In [328]:
df_prospect_prob = df_prospect.copy()

In [329]:
df_prospect_prob.drop(columns=[ 'ID_Prospect',
                                'Nome',
                                'Cidade', 
                                'Cargo', 
                                'Estado Civil',
                                'Moradia',
                                'Viagens',
                                'Restaurantes',
                                'Entretenimento',
                                'Cashback',
                                'Compras online',
                                'Farmácias',
                                'Programas de Milhagem',
                                'Postos de Combustível',
                                'Mercados',
                                'Tempo na Empresa',
                                'Pessoas em Casa', 
                                'Possui Carro'], axis=1, inplace=True)

In [330]:
predicao = final_model.predict(df_prospect_prob)
predicao = np.clip(predicao, 0, 100)

In [331]:
df_prospect['Probabilidade Inadimplencia'] = predicao

### Calcular Score - Prospect

In [332]:
def calcular_score(row):
    renda_norm = min(row['Renda']/5000,1)
    invest_norm = min(row['Investimentos']/1e6,1)
    ativos_norm = min(row['Ativos']/2e6,1)
    dividas_norm = min(row['Dívidas']/5e5,1)
    score_base = (renda_norm*0.4 +invest_norm*0.3 + ativos_norm*0.2 - dividas_norm*0.4) * 1000
    score_ajustado = score_base * (1-(row['Probabilidade Inadimplencia'] / 100))
    return int(round(max(0, min(1000, score_ajustado))))

In [333]:
df_prospect['Score'] = df_prospect.apply(calcular_score, axis=1)

In [334]:
df_prospect.head(10)

,ID_Prospect,Nome,Idade,Viagens,Restaurantes,Entretenimento,Cashback,Compras online,Farmácias,Programas de Milhagem,...,Tempo na Empresa,Pessoas em Casa,Moradia,Possui Carro,Renda,Investimentos,Ativos,Dívidas,Probabilidade Inadimplencia,Score
0,de943fb0-5e23-4e7f-8130-1aa76b33f4dc,Francisco Fonseca,24,1,1,4,5,4,1,5,...,14,6,Próprio,True,46006,736831,371505,88180,42.219,340
1,5050d0e5-5d35-4cb1-a208-596a3ee4ac1d,Melissa Mendes,58,1,4,5,3,2,3,3,...,4,2,Próprio,False,38086,374702,1828554,31206,50.168,334
2,435005c0-50ea-4755-acef-b4362c08dccd,Lucca Costa,18,2,4,1,1,3,1,3,...,20,6,Próprio,False,42531,933858,1295274,178144,76.357,158
3,4e9951b4-1790-484c-ab25-189aa9089241,Francisco da Luz,20,4,3,1,4,1,4,2,...,11,6,Próprio,True,16112,698799,1574952,434757,88.360,49
4,cb434840-2bf0-4516-8a79-8c9ada489c03,Sr. Vinícius Costela,22,1,1,4,3,5,2,3,...,0,3,Próprio,True,3059,920950,449866,224609,67.165,127
5,cbadf676-7690-4837-891d-9f9d7eb20d95,Emanuelly da Mota,20,5,5,3,1,1,5,3,...,19,5,Alugado,False,23682,796881,1572953,127732,79.874,140
6,2b07d568-893c-4199-8927-b47ce77afeef,Arthur Gabriel da Cruz,76,3,5,3,2,1,3,4,...,15,5,Alugado,True,27069,598939,898573,494353,71.752,77
7,ca81a1c4-9cc5-4519-a781-8bc6509a6e39,Benicio Barbosa,76,5,1,1,5,4,4,4,...,15,5,Próprio,True,47966,932984,456768,347189,47.162,237
8,c2fa002d-97ca-4abf-b473-098651f7dd58,Danilo Novaes,30,5,3,3,5,3,2,1,...,12,2,Próprio,False,1527,228554,336970,331813,86.871,0
9,f7a15aaf-9b2c-4094-abb7-d7882b9054e0,Rebeca Aragão,40,1,1,4,2,4,3,3,...,3,4,Próprio,True,39149,64118,1316458,97921,75.044,118


In [335]:
df_prospect.describe()

,Idade,Viagens,Restaurantes,Entretenimento,Cashback,Compras online,Farmácias,Programas de Milhagem,Postos de Combustível,Mercados,Tempo na Empresa,Pessoas em Casa,Renda,Investimentos,Ativos,Dívidas,Probabilidade Inadimplencia,Score
count,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000,500.000
mean,48.138,2.948,3.126,3.006,2.914,2.936,2.958,3.080,2.988,2.992,9.676,3.446,24382.748,492080.454,1023245.158,242693.750,65.884,158.810
std,18.117,1.429,1.435,1.430,1.428,1.394,1.370,1.455,1.430,1.416,6.085,1.689,13771.635,299296.278,614011.434,142889.781,23.980,133.264
min,18.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,1.000,1035.000,4633.000,5153.000,479.000,0.000,0.000
25%,32.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,2.000,4.000,2.000,12788.000,232767.250,454288.000,120485.000,58.702,63.000
50%,48.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,3.000,9.000,3.000,23542.000,475070.000,1047575.500,243617.000,73.944,119.000
75%,63.250,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,4.000,15.000,5.000,36373.500,770409.250,1575179.500,364049.000,81.941,212.000
max,80.000,5.000,5.000,5.000,5.000,5.000,5.000,5.000,5.000,5.000,20.000,6.000,49755.000,996338.000,1999861.000,498469.000,95.346,676.000


In [336]:
df_prospect.to_csv(r'.\datasets\dataset_prospect_score.csv', index=False)